In [ ]:
#@title init

blackurls = ['https://www.asurascans.com/wp-content/uploads/2021/05/ending-page-1.png']

try:
  import cloudscraper
except:
  !pip3 install cloudscraper
  import cloudscraper


from urllib.parse import urlparse
from google.colab import files
from bs4 import BeautifulSoup
from itertools import chain
from PIL import ImageFile
# from requests import get
from bs4 import element
from PIL import Image
import io
import shutil
import http.client
from collections import namedtuple
from IPython.display import clear_output
import requests

get_res = namedtuple('GetRes', ['content', 'status_code'])

import os

def get(*args, **kwargs):
    scraper = cloudscraper.create_scraper()
    return scraper.get(*args, **kwargs)



from http.cookies import SimpleCookie
rawdata ='wpmanga-reading-history=W3siaWQiOjY4LCJjIjoiMTAzNTIiLCJwIjoxLCJpIjoiIiwidCI6MTY2MjM2NjYwNH1d; __gpi=UID=0000097b7100ef5f:T=1662381056:RT=1662381056:S=ALNI_MYJnHzc0qqoRrzbl-_q3HT6tTmkow; _pbjs_userid_consent_data=3524755945110770; __gads=ID=e7d4017eb68ecc6d:T=1662381056:S=ALNI_MZh0n8IUw2dfYXUDjqs5-srpfsLVw; FCNEC=[["AKsRol8UoVBQAhqwOLTR5zQ0dHojTAxZ-D3J8bnBxntdCYZXNXls79EKWoEl9J2I2Xt50-lxhI6kjX3DkJ2nSnx_bK7FdN1qi8INbaA1iFg2CJIF68MWcQQ1g4EpGKN1N5RJJtHH3_o0XQYL-ZLdDsP7BijfkuWQow=="],null,[]]; _cc_id=a7a75da27ba3c7d2449890ab9e6697a8; cto_bundle=PyWgil9wTGpkOUM1UTVaNmxtT3hKR05JNzBnJTJCOWl4VWp3RnVNMFpqT0p3NElEQVM3QVp3N1BsaEVVeWFVOWozMjNQJTJCMmlTSHBoRFp2Qm5qcEFRdERQbGVUOGZMMWtVeHA4JTJCVEtBJTJCZUNVcGVDRCUyQlpWNmc4d2NZNlI0SGUwaVA4clp2NGN1eE9Hd3RIOVJDT1VnN0Ywc1hpaDVBJTNEJTNE; _ga_X600HD8CLR=GS1.1.1662385321.2.0.1662385321.0.0.0; _gid=GA1.2.1768630006.1663112839; __cf_bm=fO0xPbtNcTeecz8pH8VE5luOUD3PRqw1DoVOT.qvOzo-1663112894-0-ASgjewvb/+79BRHuMJYwpDuFBrKnT2ojzqGBtAouqfdroPC1odqWGuvsYS0aCHqIHrRekIhjd0+HNBDpZiYzF08Qs+wNAaSWdzjGUPm5+7XpxWU6A+dCVWiC7GmxIIn/7w==; _ga=GA1.1.865995110.1662381032; _ga_GPHF8WRS7G=GS1.1.1663112838.3.1.1663113158.0.0.0; PHPSESSID=fgj2dnsfu3dl10g25805k72s2u'
cookie = SimpleCookie()
cookie.load(rawdata)
cookies = {k: v.value for k, v in cookie.items()}

ImageFile.LOAD_TRUNCATED_IMAGES = True
manga_root = '/content/drive/MyDrive/manga'
addtional_paths = ['/content/drive/Shareddrives/TestDrive/manga']

def Content(url, retries=10):
  atempts = 0
  while atempts < retries:
    res = get(url)
    if res.status_code == 200:
      break
    else:
      if url in blackurls:break
      print(f'{url} returened code {res.status_code}')
    atempts += 1
    
  if atempts >= retries:
    raise Exception(f'too many retries on {url}')
  return res.content

def Soup(url):
  content = Content(url)
  soup = BeautifulSoup(content)
  return soup

def Title(soup):
  return soup.find('title').contents[0]

def Values(soup, search_args, init_filter=None):
  # first is the filter args for bs4.BS.find_all
  # last is tag atribute
  # thigs in between is for navigating the img tag
  if init_filter:
    soup = soup.find(*init_filter)
  # print(soup)
  root_eles = soup.find_all(*search_args[0])
  eles = []
  for ele in root_eles: 
    _ = ele
    for q in search_args[1:-1]:
      _ = _.find(*q)
    eles.append(_)
  if search_args[-1] is not None:
    urls = [ele[search_args[-1]].strip() for ele in eles]
  else:
    urls = [ele.contents for ele in eles]
  return urls

def series_folder(title, root=manga_root):
  directory = os.path.join(root, title)
  try:
    os.makedirs(directory)
  except FileExistsError:
    pass
  return directory

def pdf_data(title, root='.'):
  name = f"{title}.pdf"
  path = os.path.join(root, name)
  return name, path

def save_pdf(pdf_path, image_urls):
  interupted = False
  images = [Image.open(io.BytesIO(Content(url))) for url in image_urls]
  RGB_images = [img.convert('RGB') for img in images]
  try:
    RGB_images[0].save(pdf_path, save_all=True, append_images=RGB_images[1:])
  except KeyboardInterrupt:
    interupted = True
  if interupted:
    raise KeyboardInterrupt('raised deled interuption')

def Domain(url):
  return urlparse(url).netloc



class AnimeSite:
  def __init__(self, domain, **kwargs):
    self.domain = Domain(domain)
    for key, value in kwargs.items():
      setattr(self, key, value)


# replace terms:
RS = (' - Reaper Scans','') #reaperscans.com
MS1 = (' - Manga - Manhwa Read Free!', '') #mangastic.me
MS2 = (' – Manga – Manhwa Read Free!', '')

# search_terms:
RS_ch = [['img'],'src']
RS_series_old = [['div',{'class':'chapter-link'}], ['a'], 'href']
RS_series=[ ['li'],['a'],'href']
MS_ch = RS_ch
MS_series = [['li',{'class':'wp-manga-chapter '}], ['a'], 'href']

def MS_breadcrumb(soup):
  breadcrumb = Values(soup.find('ol',{'class':'breadcrumb'}),[['li'],None])
  breadcrumb = list(chain(*breadcrumb))
  breadcrumb = list(filter(lambda e:e != '\n', breadcrumb))
  # if type(breadcrumb[-1]) == element.Tag:
  #   series_name = breadcrumb[-1].contents[0].strip()
  #   chapter = None
  # else:
  #   series_name = breadcrumb[-2].contents[0].strip()
  #   chapter = breadcrumb[-1].strip()
  return breadcrumb

def MS_kind(soup):
  breadcrumb = MS_breadcrumb(soup)
  if type(breadcrumb[-1]) == element.Tag:
    return 'series'
  else:
    return 'chapter'

def RS_kind(soup):
  navs = soup.find_all('nav')
  if len(navs) > 1:
    return 'chapter'
  else:
    return 'series'
  print(navs)
  try:
    MS_kind(soup)
    return 'chapter'
  except:
    return 'series'

def MS_names(soup):
  breadcrumb = MS_breadcrumb(soup)
  series_name = breadcrumb[-2].contents[0].strip()
  chapter_name = breadcrumb[-1].strip()
  return series_name, chapter_name
  
def download_chapter(soup, site, hot_download=False):
  series_name, chapter_name = site.names(soup)
  pdf_root = series_folder(series_name)
  pdf_name, pdf_path = pdf_data(f'{series_name} - {chapter_name}', root=pdf_root)
  if os.path.exists(pdf_path):
    print(f'{pdf_name} is already there!')
    return
  
  image_urls = Values(soup, site.chapter, site.init)
  print(image_urls)
  image_urls = [i for i in image_urls if i not in blackurls]
  print(pdf_name)
  save_pdf(pdf_path, image_urls)
  for add_path in addtional_paths:
    dest = series_folder(series_name, add_path)
    shutil.copy(pdf_path, dest)

  if hot_download:
    files.download(pdf_path)

def AS_kind(soup):
  n = len(soup.find('div',{'class':'ts-breadcrumb bixbox'}).find_all('li'))
  if n == 2:
    return 'series'
  if n == 3:
    return 'chapter'
def AS_names(soup):
  full_name = soup.find('h1',{"class":"entry-title"}).contents[0].strip()
  series_name = soup.find('div',{"class","allc"}).a.contents[0].strip()
  chapter_name = full_name[len(series_name):].strip()
  return series_name, chapter_name

def mangadass_kind(soup):
  n = len(soup.find("ol", {"class":"breadcrumb"}).find_all("li"))
  if n == 2:
    return 'series'
  if n == 3:
    return 'chapter'

def mangadass_names(soup):
  names = soup.find("ol", {"class":"breadcrumb"}).find_all("li")
  series_name = names[1].find('a').contents[0].strip()
  chapter_name = names[2].find('a').contents[0].strip()
  return series_name, chapter_name

def leviatanscans_kind(soup):
  heading = soup.find_all('div', {'id':'manga-title'})
  n = len(heading)
  if n == 1:
    return 'series'
  else:
    return 'chapter'

def leviatanscans_names(soup):
  breadcrumb = soup.find('ol', {'class':'breadcrumb'})
  lis = breadcrumb.find_all('li')
  series_name = lis[-2].a.contents[0].strip()
  chapter_name = lis[-1].contents[0].strip()
  return series_name, chapter_name

AS_ch = [['img',{'loading':'lazy'}],'src']
AS_series = [['div',{'class':'eph-num'}],['a'],'href']

manga_sites = {
    'mangastic.me': AnimeSite('https://mangastic.me/',
                              replace=[MS1, MS2],
                              chapter=MS_ch,
                              series=MS_series,
                              kind=MS_kind,
                              names=MS_names,
                              init=None
                              ),
    'reaperscans.com': AnimeSite('https://reaperscans.com/',
                                 replace=[RS],
                                 chapter=RS_ch,
                                 series=RS_series,
                                 kind=RS_kind,
                                 names=MS_names,
                                 init=['p',{'class':'py-4'}],
                                 s_init=['ul',{'role':'list'}]
                                 ),
      'reincarnatedasthecrazedheir.com':AnimeSite('https://reincarnatedasthecrazedheir.com/'),
      'www.asurascans.com':AnimeSite('https://www.asurascans.com/',
                                     chapter=AS_ch,
                                     series=AS_series,
                                     kind=AS_kind,
                                     names=AS_names,
                                     init=['div',{'id':'readerarea'}]
                                     ),
      "imperfectcomic.org":AnimeSite('imperfectcomic.org',
                                     chapter=[['img'],'src'],
                                     series=AS_series,
                                     kind=AS_kind,
                                     names=AS_names,
                                     init=['div',{'id':'readerarea'}]
                                     ),
      "mangadass.com":AnimeSite("https://mangadass.com/",
                                chapter=[['img'],'data-src'],
                                series=[['li',{"class","a-h"}], ['a'], "href"],
                                kind=mangadass_kind,
                                names=mangadass_names,
                                init=['div',{"class":"read-content"}]
                                ),
      "en.leviatanscans.com":AnimeSite("https://en.leviatanscans.com/",
                                       chapter=[['img'], 'src'],
                                       series=[['a'],'href'],
                                       kind=leviatanscans_kind,
                                       names=leviatanscans_names,
                                       init=['div',{'class':'reading-content'}]),
      "mangahub.io":AnimeSite("https://mangahub.io/",
                              chapter=[['img'], 'src'],
                              init=['div',{'class':'_2aWyJ'}])
}
manga_sites['asura.gg']=manga_sites['www.asurascans.com']

clear_output()

In [ ]:
url = "https://asura.gg/reaper-of-the-drifting-moon-chapter-39/" #@param {type:"string"}
download_if_chapter_as_well = True #@param {type:"boolean"}
def process(url,hot_download=False):
  domain = Domain(url)
  site = manga_sites[domain]
  if not site:
    raise Exception(f'no records of {domain}')
  soup = Soup(url)
  kind = site.kind(soup)
  print(kind)

  if kind == 'chapter':
    while True:
      try:
        site = manga_sites.get(domain)
        if not site:
          raise Exception(f'no records of {domain}')
        soup = Soup(url)
        kind = site.kind(soup)
        if kind == 'chapter':
          print('dl')
          download_chapter(soup, site, hot_download) 
        break
      except Exception as err:
        raise err
        print('failed:retrying...')
  if kind == 'series':
    if domain == "en.leviatanscans.com":
      soup = BeautifulSoup(requests.post(url+'/ajax/chapters').content)
    chapter_links = Values(soup, site.series, getattr(site, 's_init', None))
    print(chapter_links)
    chapter_links.reverse()
    for chapter in chapter_links:
      if chapter[0] == '/':
        chapter = "https://"+domain + chapter
      process(chapter)

process(url,download_if_chapter_as_well)

chapter
dl


KeyError: ignored

In [ ]:
#@title custom schemes
custom_scheme = {
    'reincarnatedasthecrazedheir.com': [['div', {'class':"separator"}],['img'],'src'],
}

In [ ]:
#@title Custom Download
url = "https://zinmanga.com/manga/mercenary-enrollment/chapter-103/" #@param {type:"string"}
domain = Domain(url)
site = custom_scheme.get(domain)
if not site:
  raise Exception(f'no records of {domain}')
soup = Soup(url)
pdf_name = Title(soup) + '.pdf'
if os.path.exists(pdf_name):
  os.remove(pdf_name)
image_urls = Values(soup, site)
save_pdf(pdf_name, image_urls)
files.download(pdf_name)

Exception: ignored

In [ ]:
scraper = cloudscraper.create_scraper()

path = [['img'], 'src']
init = ['div',{'class':'_2aWyJ'}]

def main():
  url = "https://mangahub.io/chapter/dr-stone_111/chapter-109"
  res = scraper.get(url)
  print(res.status_code)
  if not res.ok:return
  soup = BeautifulSoup(res.content)
  data = Values(soup, path, init)
  print(data)
main()

In [ ]:
scraper.get('https://img.mghubcdn.com/file/imghub/dr-stone/109/1.jpg')